<a href="https://colab.research.google.com/github/NancyFulda/towards-neural-programming-interfaces/blob/colab/notebooks/politics/politics_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Notebook training NPI on political views

This notebook is also an example on how to use the NPI project to train your own NPI in a way that can affect GPT2's sentiment or emotion. This notebook attempts to train the GPT2 model with NPI to possess a specific political view.

## Setup

### Prerequisite setup:

*   Ensure you selected a runtime with a GPU. You can do that by clicking Runtime -> Change runtime type ->  Hardware Accelerator -> GPU
*   After running the setup code in this section, restart the runtime by clicking Runtime -> Restart runtime.

After you have done the prerequisite setup, you will not need to rerun this section again.


In [9]:
!cd towards-neural-programming-interfaces && git pull

remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 13 (delta 9), reused 13 (delta 9), pack-reused 0
Unpacking objects: 100% (13/13), done.
From https://github.com/NancyFulda/towards-neural-programming-interfaces
   1771ff3..8443bb7  colab      -> origin/colab
Updating 1771ff3..8443bb7
Fast-forward
 install_dependencies.sh                      |  4 ----
 notebooks/politics/politics.ipynb            | 17 +++++++++++------
 src/npi/dataset/construct_dataset.py         |  2 +-
 src/npi/dataset/npi_dataset.py               |  8 ++++----
 src/npi/training/style_classifier_trainer.py |  3 ---
 5 files changed, 16 insertions(+), 18 deletions(-)


In [1]:
!git clone https://github.com/NancyFulda/towards-neural-programming-interfaces.git
!cd towards-neural-programming-interfaces && ./install_dependencies.sh

Cloning into 'towards-neural-programming-interfaces'...
remote: Enumerating objects: 427, done.
remote: Counting objects: 100% (356/356), done.
remote: Compressing objects: 100% (253/253), done.
remote: Total 427 (delta 194), reused 218 (delta 83), pack-reused 71
Receiving objects: 100% (427/427), 305.29 MiB | 14.21 MiB/s, done.
Resolving deltas: 100% (216/216), done.
Checking out files: 100% (60/60), done.
Branch 'colab' set up to track remote branch 'colab' from 'origin'.
Switched to a new branch 'colab'
installing packages
Obtaining file:///content/towards-neural-programming-interfaces (from -r requirements.txt (line 2))
     |████████████████████████████████| 753.2 MB 14 kB/s 
     |████████████████████████████████| 6.6 MB 14.7 MB/s 
     |████████████████████████████████| 47 kB 4.7 MB/s 
     |████████████████████████████████| 132 kB 43.3 MB/s 
     |████████████████████████████████| 895 kB 44.9 MB/s 
     |████████████████████████████████| 8.6 MB 43.6 MB/s 
     |████████████████

In [2]:
# Download the NewB news source dataset
!git clone https://github.com/JerryWei03/NewB.git

Cloning into 'NewB'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 14 (delta 1), reused 4 (delta 1), pack-reused 0
Unpacking objects: 100% (14/14), done.


### Post setup

***Note: You will need to restart the runtime on Colab to continue***

Click on Runtime -> Restart Runtime to do so.


In [1]:
# For dev purposes, enable autoreload of modules
%load_ext autoreload

In [10]:
import pandas as pd
import torch

%autoreload

# Constructing training dataset
from npi.dataset import NPIDatasetConstructor, NPIDatasetLoader

# Models
from npi.models import NPITrainingModels

# Training NPI
from npi.training import NPIStyleTrainer, NPITrainer

# Configuration
from npi.config import NPIConfig


Note that each step in training an NPI is conditioned on the variable toggles, so you can disable certain steps to not redo them again when running the whole notebook.

In [3]:
# Toggle steps
construct = True
train_class = True
test_class = True
train_npi = True


In [4]:
model_layers = [5, 11]

device = torch.device(f"cuda:0")
config = NPIConfig(
    device, gpt_model="gpt2", perturbation_indices=model_layers, npi_name="politics"
)


## Create Dataset

### Get Data from NewB GitHub repo

See `construct_politics_data` to see how to construct data.

In [ ]:
if construct:
    with open("./NewB/train_orig.txt", "r", newline="") as f:
        data = pd.read_csv(f, delimiter="\t", names=["view", "text"])
        data["view"] = data["view"].replace(
            to_replace={0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1}
        )
        data = data[data.view != 5]
        data = data.sample(frac=1)
        data_iter = zip(data["text"], data["view"])
        construct_data = NPIDatasetConstructor(config)
        construct_data.construct_dataset(data_iter, 4000) # Only creating 4000 training data points


## Train classifier from Dataset



In [ ]:
model_loader = NPITrainingModels(config)
dataset_loader = NPIDatasetLoader(config, target="style")
style_trainer = NPIStyleTrainer(config, class_lr=1e-5)
classifier = None
if train_class:
    classifier = style_trainer.train_classifier(
        model_loader,
        dataset_loader,
        num_epochs=5,
        #continue_epoch=4,  # Set epoch of classifier to load to continue training.
    )


### Test Classifier


In [ ]:
_, test_loader, _, _ = dataset_loader.load_train_and_test_dataloaders()

classifier = model_loader.load_style_classifier() if not classifier else classifier
# When output=True, it will log the following in models/npi_models/style_classifier_output.log:
#   (generated text from the dataset)
#   truth={target_label} actual={model_output}
style_trainer.test_model(test_loader, classifier, torch.nn.BCELoss(), output=True)


Classifier INIT
Defining classifier model
Loading style_model weights from models/npi_models/politics_style_model_008.pth


(69738789.6, 0.44000000000000006)

## Training NPI

In [ ]:
# Initialize models for training and testing
models = NPITrainingModels(
    config,
)

In [ ]:
num_epochs = 6
split_ratio = 0.25
batch_size = 5
headstart = 0  # set at 0 for debugging

if train_npi:
    trainer = NPITrainer(config, batch_size=batch_size, headstart=headstart)
    dataset_loader = NPIDatasetLoader(config, split_ratio=split_ratio)
    trainer.train_adversarial_npi(models, num_epochs, dataset_loader)


## Test NPI

Using the model loader, you can directly generate NPI text.


In [ ]:
text = "Trump today was found"
print(F"Original GPT2 output: '{text}'\n{models.gpt2_generate_text(text)}")


Initializing GPT2WithNPI model with tokenizer -- not being placed on GPU until npi loss evaluation



Original GPT2 output: 'Trump today was found'
 to have been a liar.

The FBI has been investigating the Trump campaign's ties to Russia.

The Trump campaign has also been accused of using the word "radical" in a tweet.

"I am not a fan of the idea of a "big-screen" movie.

"I'm not going to say that I'm going to say that I'm going to say that I'm going to say that I'm going to say that I'm going to say that I'm going to say


In [ ]:
print(F"NPI GPT2 output: '{text}'\n{models.npi_generate_text(text)}")

NPI INITIALIZATION
Loading npi_model weights from models/npi_models/politics_npi_model_000.pth


NPI GPT2 output: 'Trump today was found'
 to have been a liar.

The FBI has been investigating the Trump campaign's ties to Russia.

The Trump campaign has also been accused of using the word "radical" in a tweet.

"I am not a racist, I am not a racist, I am not a racist, I am not a racist, I am not a racist, I am not a racist, I am not a racist, I am not a racist, I am not a racist, I am not a racist, I am
